In [1]:
import numpy as np
import pandas as pd
import csv

In [2]:
cacontents = pd.read_excel('casesc.xlsx')
#cacontents.head(3)

In [4]:
df = pd.DataFrame(cacontents)
print(df.shape)

(74611, 9)


In [19]:
import urllib.request
import pandas
import json
import time


def get_access_token():
    """
    获取百度AI平台的Access Token
    """
    # 使用你的API Key及Secret Key
    host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=FIGcoqTMBM95h0GlZQneRz8U&client_secret=qQs3qaPmSIlEkIPw3fIcyf0EGp2rfTXD'
    request = urllib.request.Request(host)
    request.add_header('Content-Type', 'application/json; charset=UTF-8')
    response = urllib.request.urlopen(request)
    content = response.read().decode('utf-8')
    rdata = json.loads(content)
    return rdata['access_token']

In [20]:
def sentiment_classify(text, acc):
    """
    获取文本的感情偏向（消极 or 积极 or 中立）
    参数：
    text:str 本文
    """
    raw = {"text":"内容"}
    raw['text'] = text
    data = json.dumps(raw).encode('utf-8')
    # 情感倾向分析接口
    host = "https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?charset=UTF-8&access_token=" + acc
    request = urllib.request.Request(url=host, data=data)
    request.add_header('Content-Type', 'application/json')
    response = urllib.request.urlopen(request)
    content = response.read().decode('utf-8')
    rdata = json.loads(content)
    return rdata

In [21]:
# 获取access_token
access_token = get_access_token()

In [ ]:
# 输出情感极性分类结果,0:负向,1:中性,2:正向
sentiments = []
for text in df['CONTENT']:
    time.sleep(1)#速度太快会出错
    result = sentiment_classify(str(text), access_token)
    value = result['items'][0]['sentiment']
    sentiments.append(value)
    print(value,text)
    #为了防止突然中断，将以出来的分值
    with open('texttxt.csv','a',newline='',encoding='gb18030') as csvfile:
        writer  = csv.writer(csvfile)
        writer.writerow([value,text])
            

    # 添加评情感列
df['emotional'] = sentiments
#print(df.head())

#导出至excel
df.to_excel('emotional19439.xlsx',index=False, header=None,encoding='utf_8_sig')#,encoding="gb18030"
